In [5]:
import json
import os
import sys
import math
import datetime
import pathlib

dir_path = pathlib.Path(os.path.abspath('')).resolve()
print(dir_path)
HOME_DIR = str(dir_path).split('/OpenFormat')[0]

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

PROJ_SRC_DIR = f'{HOME_DIR}/OpenFormat/OpenFormat-Public'
sys.path.insert(1, f'{PROJ_SRC_DIR}')
from python.scripts.utils import *

/home/haoyuq/EvaluationOfColumnarFormats/OpenFormat/OpenFormat-Public/python/encoding_size


In [8]:
num_rows = 1000 * 1000
num_cols = 20
sort_list = [
                0.01,
                0.2,
                0.4,
                0.6,
                0.8,
                1.0,
                1.2,
                1.4,
                1.6,
                1.8,
                2.0
            ]
type_list = ['int', 'float', 'string']
def width_list(t):
    if t == 'string':
        sort_list = [
                    5,
                    10,
                    25,
                    50,
                    100,
                    250,
                    500,
                    # 1000
                ]
    elif t == 'int':
        sort_list = [
            10,
            100,
            1000,
            10000,
            100000,
            1000000,
            10000000,
            100000000
        ]
    elif t == 'float':
        sort_list = [
            100,
            10000,
            1000000,
            100000000
        ]
    else:
        assert False
    return sort_list

In [3]:
num_rows = 1000 * 1000
num_cols = 20
sort_list = [
                0.01,
                0.2,
                0.4,
                0.6,
                0.8,
                1.0,
                1.2,
                1.4,
                1.6,
                1.8,
                2.0
            ]
type_list = ['string']
def width_list(t):
    if t == 'string':
        sort_list = [
                    250,
                    500,
                    # 1000
                ]
    elif t == 'int':
        sort_list = [
            10,
            100,
            1000,
            10000,
            100000,
            1000000,
            10000000,
            100000000
        ]
    elif t == 'float':
        sort_list = [
            100,
            10000,
            1000000,
            100000000
        ]
    else:
        assert False
    return sort_list

In [ ]:
%%time
# generate csv files
os.chdir(f'{PROJ_SRC_DIR}/benchmark/generator_v2')
!pwd
for t in type_list:
    print(t)
    sort_list = width_list(t)
    for s in sort_list:
        print(s)
        base_name = f'{t}_r{num_rows}_c{num_cols}_width{s}'
        # if os.path.exists(f'{dir_path}/{base_name}'):
        #     continue
        os.system(f'python3 gen_workloads.py \
            {t} {num_rows} {num_cols} {base_name} width {s}')
        os.system(f'cp -r {base_name} {dir_path} && \
            rm -r {base_name}')
os.chdir(dir_path)
!pwd


/home/haoyuq/EvaluationOfColumnarFormats/OpenFormat/OpenFormat-Public/benchmark/generator_v2
string
250


In [6]:
pq_config = enumerate_config(f'{PROJ_SRC_DIR}/python/experiments/pq_encoding.json')
orc_config = enumerate_config(f'{PROJ_SRC_DIR}/python/experiments/orc_encoding.json')
pq_name_suffix = '_' + "_".join([str(i) for i in list(pq_config[0].values())]) + '.parquet'
orc_name_suffix = '_' + "_".join([str(i) for i in list(orc_config[0].values())]) + '.orc'
print(pq_config)

loading config from /home/haoyuq/EvaluationOfColumnarFormats/OpenFormat/OpenFormat-Public/python/experiments/pq_encoding.json ...
loading config from /home/haoyuq/EvaluationOfColumnarFormats/OpenFormat/OpenFormat-Public/python/experiments/orc_encoding.json ...
[{'compression': 'NONE', 'use_dictionary': True, 'dictionary_pagesize_limit': 2147483647, 'version': '2.6', 'data_page_version': '2.0'}]


In [7]:
# generate parquet and orc files
my_password = ''
os.chdir(f'{PROJ_SRC_DIR}/python')
for t in type_list:
    sort_list = width_list(t)
    for s in sort_list:
        base_name = f'{t}_r{num_rows}_c{num_cols}_width{s}'
        if os.path.exists(f'{dir_path}/{base_name}.orc'):
            continue
        os.system(f'cp {dir_path}/{base_name}/gen_data/{base_name}.csv data_gen')
        os.system(f'echo {my_password} | sudo -S /home/haoyuq/miniconda3/envs/gen/bin/python scripts/parquet_exp.py {base_name} pq_encoding -r=false -o=false')
        os.system(f'echo {my_password} | sudo -S /home/haoyuq/miniconda3/envs/gen/bin/python scripts/orc_exp.py {base_name} orc_encoding -r=false -o=false')
        # orc_schema = open(f'encoding/{base_name}/gen_data/{base_name}_orc_schema.txt', 'r').read()
        # os.system(f'{HOME_DIR}/orc/build/tools/src/csv-import \"{orc_schema}\" \
        #         encoding/{base_name}/gen_data/{base_name}.csv {base_name}.orc')
        os.system(f'mv {base_name}{pq_name_suffix} {dir_path}/{base_name}.parquet')
        os.system(f'mv {base_name}{orc_name_suffix} {dir_path}/{base_name}.orc')
        # os.system(f'mv {base_name}.orc encoding/{base_name}.orc')
os.chdir(dir_path)

[sudo] password for haoyuq: rm: cannot remove 'outputs/stats.json': No such file or directory


loading config from ./experiments/pq_encoding.json ...
----Running experiment: 1 out of 1----
NONE_True_2147483647_2.6_2.0
write parquet time (s): 9.367289066314697


[sudo] password for haoyuq: 

loading config from ./experiments/orc_encoding.json ...
----Running experiment:----
UNCOMPRESSED_speed_1_2147486647
write orc time (s): 45.483893156051636


[sudo] password for haoyuq: 

loading config from ./experiments/pq_encoding.json ...
----Running experiment: 1 out of 1----
NONE_True_2147483647_2.6_2.0
write parquet time (s): 18.147418975830078


[sudo] password for haoyuq: 

loading config from ./experiments/orc_encoding.json ...
----Running experiment:----
UNCOMPRESSED_speed_1_2147486647
write orc time (s): 46.684802532196045


In [10]:
import yaml
os.chdir(f'{PROJ_SRC_DIR}/python')
for t in type_list:
    sort_list = width_list(t)
    for s in sort_list:
        base_name = f'{t}_r{num_rows}_c{num_cols}_width{s}'
        arrow_schema_name = f'encoding_size/{base_name}/gen_data/{base_name}_arrow_schema.txt'
        with open(arrow_schema_name, 'r') as f:
            lines = f.readlines()
        columns = []
        cnt = 0
        for line in lines:
            if 'message' in line or '}' in line or len(line)==0:
                print(line)
                continue
            col_name = f'col{cnt}'
            col_type = None
            if 'String' in line:
                col_type = 'string'
            elif 'int' in line:
                col_type = 'integer'
            elif 'double' in line:
                col_type = 'double'
            if col_type is None:
                print(line)
                col_type = 'skip'
            cnt += 1
            columns.append({'name': col_name, 'type':col_type})
        with open(f'encoding_size/{base_name}/gen_data/{base_name}_btrblocks_schema.yaml','w') as f:
            yaml.dump({'columns':columns}, f)

message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}


In [11]:
os.chdir(f'{PROJ_SRC_DIR}/python')
for t in type_list:
    sort_list = width_list(t)
    for s in sort_list:
        base_name = f'{t}_r{num_rows}_c{num_cols}_width{s}'
        os.system(f'mkdir encoding_size/{base_name}/gen_data/binary')
        os.system(f'mkdir encoding_size/{base_name}/gen_data/btr')
    
        os.system(f'~/btrblocks/build/csvtobtr -create_binary -binary encoding_size/{base_name}/gen_data/binary/ -csv encoding_size/{base_name}/gen_data/{base_name}.csv -yaml encoding_size/{base_name}/gen_data/{base_name}_btrblocks_schema.yaml')
        os.system(f'~/btrblocks/build/csvtobtr -create_btr -btr encoding_size/{base_name}/gen_data/btr/ -binary encoding_size/{base_name}/gen_data/binary/ -csv encoding_size/{base_name}/gen_data/{base_name}.csv -yaml encoding_size/{base_name}/gen_data/{base_name}_btrblocks_schema.yaml')

[2024-03-06 17:26:53.171] [info] Creating binary files in encoding_size/int_r1000000_c20_width10/gen_data/binary/
column name = col0 , type = integer
column name = col1 , type = integer
column name = col2 , type = integer
column name = col3 , type = integer
column name = col4 , type = integer
column name = col5 , type = integer
column name = col6 , type = integer
column name = col7 , type = integer
column name = col8 , type = integer
column name = col9 , type = integer
column name = col10 , type = integer
column name = col11 , type = integer
column name = col12 , type = integer
column name = col13 , type = integer
column name = col14 , type = integer
column name = col15 , type = integer
column name = col16 , type = integer
column name = col17 , type = integer
column name = col18 , type = integer
column name = col19 , type = integer
[haoyu.debug.Parser] writing binary fileencoding_size/int_r1000000_c20_width10/gen_data/binary/1_col0.bitmap
Writing binary file : encoding_size/int_r100000

fd != -1: No such file or directory
csvtobtr: /home/haoyuq/btrblocks/btrblocks/storage/MMapVector.hpp:47: void btrblocks::mmapvector::Vector<T>::readBinary(const char*) [with T = int]: Assertion `false' failed.
Aborted (core dumped)
fd != -1: No such file or directory
csvtobtr: /home/haoyuq/btrblocks/btrblocks/storage/MMapVector.hpp:47: void btrblocks::mmapvector::Vector<T>::readBinary(const char*) [with T = int]: Assertion `false' failed.
Aborted (core dumped)


[2024-03-06 17:28:06.620] [info] Creating binary files in encoding_size/float_r1000000_c20_width100/gen_data/binary/
column name = col0 , type = double
column name = col1 , type = double
column name = col2 , type = double
column name = col3 , type = double
column name = col4 , type = double
column name = col5 , type = double
column name = col6 , type = double
column name = col7 , type = double
column name = col8 , type = double
column name = col9 , type = double
column name = col10 , type = double
column name = col11 , type = double
column name = col12 , type = double
column name = col13 , type = double
column name = col14 , type = double
column name = col15 , type = double
column name = col16 , type = double
column name = col17 , type = double
column name = col18 , type = double
column name = col19 , type = double
[haoyu.debug.Parser] writing binary fileencoding_size/float_r1000000_c20_width100/gen_data/binary/1_col0.bitmap
Writing binary file : encoding_size/float_r1000000_c20_width1

!!! compressed is larger than raw: 
for : values - scheme = DICT difference = 5. Falling back to uncompressed.
!!! compressed is larger than raw: 
for : ? - scheme = PSEUDODECIMAL difference = 1000. Falling back to uncompressed.


[2024-03-06 17:28:27.428] [info] Creating binary files in encoding_size/float_r1000000_c20_width10000/gen_data/binary/
column name = col0 , type = double
column name = col1 , type = double
column name = col2 , type = double
column name = col3 , type = double
column name = col4 , type = double
column name = col5 , type = double
column name = col6 , type = double
column name = col7 , type = double
column name = col8 , type = double
column name = col9 , type = double
column name = col10 , type = double
column name = col11 , type = double
column name = col12 , type = double
column name = col13 , type = double
column name = col14 , type = double
column name = col15 , type = double
column name = col16 , type = double
column name = col17 , type = double
column name = col18 , type = double
column name = col19 , type = double
[haoyu.debug.Parser] writing binary fileencoding_size/float_r1000000_c20_width10000/gen_data/binary/1_col0.bitmap
Writing binary file : encoding_size/float_r1000000_c20_wi

!!! compressed is larger than raw: 
for : ? - scheme = PSEUDODECIMAL difference = 1000. Falling back to uncompressed.
!!! compressed is larger than raw: 
for : ? - scheme = PSEUDODECIMAL difference = 1000. Falling back to uncompressed.
!!! compressed is larger than raw: 
for : ? - scheme = PSEUDODECIMAL difference = 1000. Falling back to uncompressed.


[2024-03-06 17:29:09.221] [info] Creating binary files in encoding_size/float_r1000000_c20_width100000000/gen_data/binary/
column name = col0 , type = double
column name = col1 , type = double
column name = col2 , type = double
column name = col3 , type = double
column name = col4 , type = double
column name = col5 , type = double
column name = col6 , type = double
column name = col7 , type = double
column name = col8 , type = double
column name = col9 , type = double
column name = col10 , type = double
column name = col11 , type = double
column name = col12 , type = double
column name = col13 , type = double
column name = col14 , type = double
column name = col15 , type = double
column name = col16 , type = double
column name = col17 , type = double
column name = col18 , type = double
column name = col19 , type = double
[haoyu.debug.Parser] writing binary fileencoding_size/float_r1000000_c20_width100000000/gen_data/binary/1_col0.bitmap
Writing binary file : encoding_size/float_r100000

terminate called after throwing an instance of 'std::logic_error'
  what():  chunks with compressed size greater than part_size_threshold unsupported
Aborted (core dumped)


[2024-03-06 17:35:51.484] [info] Creating binary files in encoding_size/string_r1000000_c20_width500/gen_data/binary/
column name = col0 , type = string
column name = col1 , type = string
column name = col2 , type = string
column name = col3 , type = string
column name = col4 , type = string
column name = col5 , type = string
column name = col6 , type = string
column name = col7 , type = string
column name = col8 , type = string
column name = col9 , type = string
column name = col10 , type = string
column name = col11 , type = string
column name = col12 , type = string
column name = col13 , type = string
column name = col14 , type = string
column name = col15 , type = string
column name = col16 , type = string
column name = col17 , type = string
column name = col18 , type = string
column name = col19 , type = string
[haoyu.debug.Parser] writing binary fileencoding_size/string_r1000000_c20_width500/gen_data/binary/1_col0.bitmap
Writing binary file : encoding_size/string_r1000000_c20_wid

terminate called after throwing an instance of 'std::logic_error'
  what():  chunks with compressed size greater than part_size_threshold unsupported
Aborted (core dumped)


In [12]:
# collect size
os.chdir(dir_path)
os.system('rm outputs/stats.json')

def get_dir_size(path='.'):
    total = 0
    with os.scandir(path) as it:
        for entry in it:
            if entry.is_file():
                total += entry.stat().st_size
            elif entry.is_dir():
                total += get_dir_size(entry.path)
    return total

for t in type_list:
    sort_list = width_list(t)
    for s in sort_list:
        base_name = f'{t}_r{num_rows}_c{num_cols}_width{s}'
        output_stats = {}
        output_stats['type'] = t
        output_stats['feature'] = 'width'
        output_stats['param'] = s
        output_stats['file'] = 'parquet'
        output_stats['size'] = os.path.getsize(f'{base_name}.parquet')
        parse_output(output_stats)
        output_stats['file'] = 'orc'
        output_stats['size'] = os.path.getsize(f'{base_name}.orc')
        parse_output(output_stats)
        output_stats['file'] = 'btrblocks'
        output_stats['size'] = get_dir_size(f'{base_name}/gen_data/btr')
        parse_output(output_stats)
collect_results()
os.system('mv outputs/stats.csv ../outputs/{}_{}.csv'.format('encoding_size_width', timestamp))

0